# Fear Behavior Implementation

**Step 1: Create display widgets**

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='30%')
display_depth = widgets.Image(format='jpeg', width='30%')
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout)
display(sidebyside)

**Step 2: Initialize camera and implement fear behavior**

The robot will retreat at different speeds based on detected obstacle distance:
- < 500mm: Fast retreat (speed 0.6)
- < 900mm: Medium retreat (speed 0.4)
- < 1300mm: Slow retreat (speed 0.2)
- >= 1300mm: Stop (safe distance)

In [ ]:
import cv2
import numpy as np
import pyzed.sl as sl
import math
import threading
import time
import motors

#initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera():
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU) 

    def _capture_frames(self):

        while(self.thread_runnning_flag==True):
           
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                scale = 0.1
                resized_image = cv2.resize(self.color_value, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
                cv2.circle(resized_image, (int(self.width*scale//2),int(self.height*scale//2)), 1, (0, 255, 0))
                display_color.value = bgr8_to_jpeg(resized_image)
                
                self.depth_image = np.asanyarray(self.depth.get_data())

                depth_image_test = self.depth_image.copy()
                depth_image_test = np.nan_to_num(depth_image_test, nan=0.0).astype(np.float32)
                
                # Define region of interest - ignore edges and ground
                depth_image_test[:120,:] = 0    # Remove top
                depth_image_test[250:,:] = 0    # Remove bottom (ground/robot body)
                depth_image_test[:,:200] = 0    # Remove left
                depth_image_test[:,472:] = 0    # Remove right

                # Filter depth values
                depth_image_test[depth_image_test < 300] = 0    # Ignore very close (robot body)
                depth_image_test[depth_image_test > 1500] = 0   # Reduce max range
                
                depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image_test, alpha=0.03), cv2.COLORMAP_JET)
                
                if(depth_image_test.max() == 0):
                    robot.stop()
                    cv2.putText(depth_colormap, 'SAFE - No threat', (150,188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                    print('SAFE - No threat detected', end='\r')
                else:
                    distance = depth_image_test[depth_image_test!=0].min()
                    
                    # FEAR BEHAVIOR - graduated response based on distance
                    if(distance < 500):
                        robot.backward(0.6)
                        cv2.putText(depth_colormap, f'DANGER {distance:.0f}mm', (150,188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
                        print(f'DANGER {distance:.0f}mm - Fast retreat', end='\r')
                    elif(distance < 900):
                        robot.backward(0.4)
                        cv2.putText(depth_colormap, f'CAUTION {distance:.0f}mm', (150,188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 165, 255), 2, cv2.LINE_AA)
                        print(f'CAUTION {distance:.0f}mm - Medium retreat', end='\r')
                    elif(distance < 1300):
                        robot.backward(0.2)
                        cv2.putText(depth_colormap, f'ALERT {distance:.0f}mm', (150,188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)
                        print(f'ALERT {distance:.0f}mm - Slow retreat', end='\r')
                    else:
                        robot.stop()
                        cv2.putText(depth_colormap, f'SAFE {distance:.0f}mm', (150,188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                        print(f'SAFE {distance:.0f}mm', end='\r')
                    
                resized_depth_colormap=cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
                display_depth.value = bgr8_to_jpeg(resized_depth_colormap)
                
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag=True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg',value)[1])

camera = Camera()
camera.start()

**To stop the camera and robot:**

In [ ]:
camera.stop()